# Limpieza del dataset Censo de Poblacion y Vivienda 2010

## 1. Introduccion
El Censo de Población y vivienda contiene el siguiente parámetro para la PCCS:
    
ID|Descripcion
--|-----------
P0309|Total de viviendas

## 2. Descarga de datos

In [1]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import zipfile
import csv
import pprint
import re

In [2]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


Existen tres maneras de descargar la información:

1. **Datos para la República Mexicana**, con la ventaja de que es un solo archivo con variables procesadas y con la desventaja de que su nivel de desagregación es estatal.

2. **Principales resultados por AGEB**, con la ventaja de que cuentan con desagregacion a nivel AGEB (Que puede agregarse a nivel Localidad, municipal y estatal) y con las desventajas de que la información está fragmentada en muchos archivos pues hay un archivo por estado, y que la informacion tiene una gran complejidad por el nivel de detalle con el que cuenta lo que también la hace más pesada.

3. **Principales resultados por LOCALIDAD**, con la ventaja de que cuentan con desagregacion a nivel localidad (Que puede agregarse a nivel municipal y estatal) y con la desventaja de que la información está fragmentada en muchos archivos pues hay un archivo por estado.

Se descargarán los datos por localidad (3)

In [3]:
# LIGAS PARA DESCARGA DE ARCHIVOS
# Las ligas para descarga tienen una raiz URL común que cambia 
# dependiendo del indicador y estado que se busque descargar 

raiz = r'http://www.beta.inegi.org.mx/contenidos/proyectos/ccpv/2010/microdatos/iter/'
links = {
    '01' : raiz+'01_aguascalientes_2010_iter_zip.zip',
    '02' : raiz+'02_baja_california_2010_iter_zip.zip',
    '03' : raiz+'03_baja_california_sur_2010_iter_zip.zip',
    '04' : raiz+'04_campeche_2010_iter_zip.zip',
    '05' : raiz +'05_coahuila_2010_iter_zip.zip',
    '06' : raiz +'06_colima_2010_iter_zip.zip',
    '07' : raiz +'07_chiapas_2010_iter_zip.zip',
    '08' : raiz +'08_chihuahua_2010_iter_zip.zip',
    '09' : raiz +'09_distrito_federal_2010_iter_zip.zip',
    '10' : raiz +'10_durango_2010_iter_zip.zip',
    '11' : raiz +'11_guanajuato_2010_iter_zip.zip',
    '12' : raiz +'12_guerrero_2010_iter_zip.zip',
    '13' : raiz +'13_hidalgo_2010_iter_zip.zip',
    '14' : raiz +'14_jalisco_2010_iter_zip.zip',
    '15' : raiz +'15_mexico_2010_iter_zip.zip',
    '16' : raiz +'16_michoacan_2010_iter_zip.zip',
    '17' : raiz +'17_morelos_2010_iter_zip.zip',
    '18' : raiz +'18_nayarit_2010_iter_zip.zip',
    '19' : raiz +'19_nuevo_leon_2010_iter_zip.zip',
    '20' : raiz +'20_oaxaca_2010_iter_zip.zip',
    '21' : raiz +'21_puebla_2010_iter_zip.zip',
    '22' : raiz +'22_queretaro_2010_iter_zip.zip',
    '23' : raiz +'23_quintana_roo_2010_iter_zip.zip',
    '24' : raiz +'24_san_luis_potosi_2010_iter_zip.zip',
    '25' : raiz +'25_sinaloa_2010_iter_zip.zip',
    '26' : raiz +'26_sonora_2010_iter_zip.zip',
    '27' : raiz +'27_tabasco_2010_iter_zip.zip',
    '28' : raiz +'28_tamaulipas_2010_iter_zip.zip',
    '29' : raiz +'29_tlaxcala_2010_iter_zip.zip',
    '30' : raiz +'30_veracruz_2010_iter_zip.zip',
    '31' : raiz +'31_yucatan_2010_iter_zip.zip',
    '32' : raiz +'32_zacatecas_2010_iter_zip.zip'
}

Key : CVE_EDO, val = Archivo

In [4]:
# Descarga de archivos a carpeta local
destino = r'D:\PCCS\00_RawData\01_CSV\CPV2010'
archivos = {}   # Diccionario para guardar memoria de descarga
for CVE_EDO, archivo in links.items():
    archivo_local = destino + r'\{}.zip'.format(CVE_EDO)
    if os.path.isfile(archivo_local):
        print('Ya existe el archivo: {}'.format(archivo_local))
        archivos[CVE_EDO] = archivo_local
    else:
        print('Descargando {} ... ... ... ... ... '.format(archivo_local))
        urllib.request.urlretrieve(archivo, archivo_local) #
        archivos[CVE_EDO] = archivo_local
        print('se descargó {}'.format(archivo_local))

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\01.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\02.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\03.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\04.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\05.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\06.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\07.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\08.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\09.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\10.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\11.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\12.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\13.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\14.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010\15.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CPV2010

In [5]:
archivos

{'01': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\01.zip',
 '02': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\02.zip',
 '03': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\03.zip',
 '04': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\04.zip',
 '05': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\05.zip',
 '06': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\06.zip',
 '07': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\07.zip',
 '08': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\08.zip',
 '09': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\09.zip',
 '10': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\10.zip',
 '11': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\11.zip',
 '12': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\12.zip',
 '13': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\13.zip',
 '14': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\14.zip',
 '15': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\15.zip',
 '16': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\16.zip',
 '17': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\17.zip',
 '18': 'D:\\PCCS\\00_RawData\\01_CSV\\CPV2010\\1

Se guardó un diccionario con k: CVE_EDO y v: Archivo comprimido local
Ahora hay que descomprimir los archivos

In [6]:
# Descompresión de archivos de m_parametro
unzipped = {}
for CVE_EDO, comprimido in archivos.items():
    target = destino + '\\' + CVE_EDO
    if os.path.isfile(target):
        print('Ya existe el archivo: {}'.format(target))
        unzipped[CVE_EDO] = target
    else:
        print('Descomprimiendo {} ... ... ... ... ... '.format(target))
        descomprimir = zipfile.ZipFile(comprimido, 'r')
        descomprimir.extractall(target)
        descomprimir.close
        unzipped[CVE_EDO] = target

Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\01 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\02 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\03 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\04 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\05 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\06 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\07 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\08 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\09 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\10 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\11 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\12 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\CPV2010\13 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_

In [7]:
for CVE_EDO, path in unzipped.items():
    print(CVE_EDO)
    x = 0
    for file in os.listdir(path):
        print(r'[{}]\\{}'.format(x, file))
        x += 1

01
[0]\\ITER_01DBF10
[1]\\ITER_01XLS10
02
[0]\\ITER_02DBF10
[1]\\ITER_02XLS10
03
[0]\\ITER_03DBF10
[1]\\ITER_03XLS10
04
[0]\\ITER_04DBF10
[1]\\ITER_04XLS10
05
[0]\\ITER_05DBF10
[1]\\ITER_05XLS10
06
[0]\\ITER_06DBF10
[1]\\ITER_06XLS10
07
[0]\\ITER_07DBF10
[1]\\ITER_07XLS10
08
[0]\\ITER_08DBF10
[1]\\ITER_08XLS10
09
[0]\\ITER_09DBF10
[1]\\ITER_09XLS10
10
[0]\\ITER_10DBF10
[1]\\ITER_10XLS10
11
[0]\\ITER_11DBF10
[1]\\ITER_11XLS10
12
[0]\\ITER_12DBF10
[1]\\ITER_12XLS10
13
[0]\\ITER_13DBF10
[1]\\ITER_13XLS10
14
[0]\\ITER_14DBF10
[1]\\ITER_14XLS10
15
[0]\\ITER_15DBF10
[1]\\ITER_15XLS10
16
[0]\\ITER_16DBF10
[1]\\ITER_16XLS10
17
[0]\\ITER_17DBF10
[1]\\ITER_17XLS10
18
[0]\\ITER_18DBF10
[1]\\ITER_18XLS10
19
[0]\\ITER_19DBF10
[1]\\ITER_19XLS10
20
[0]\\ITER_20DBF10
[1]\\ITER_20XLS10
21
[0]\\ITER_21DBF10
[1]\\ITER_21XLS10
22
[0]\\ITER_22DBF10
[1]\\ITER_22XLS10
23
[0]\\ITER_23DBF10
[1]\\ITER_23XLS10
24
[0]\\ITER_24DBF10
[1]\\ITER_24XLS10
25
[0]\\ITER_25DBF10
[1]\\ITER_25XLS10
26
[0]\\ITER_26DBF10
[1]\

Cada directorio tiene 2 carpetas, una con un archivo DBF y otra con un xls. Vamos a utilizar el xls.

In [8]:
xlsfiles = {}
for CVE_EDO, path in unzipped.items():
    print(CVE_EDO)
    xlsname = os.listdir(path)[1]
    xlspath = path+'\\'+xlsname+'\\'+xlsname+'.xls'
    xlsfiles[CVE_EDO] = xlspath
    print('{}\n'.format(xlspath))

01
D:\PCCS\00_RawData\01_CSV\CPV2010\01\ITER_01XLS10\ITER_01XLS10.xls

02
D:\PCCS\00_RawData\01_CSV\CPV2010\02\ITER_02XLS10\ITER_02XLS10.xls

03
D:\PCCS\00_RawData\01_CSV\CPV2010\03\ITER_03XLS10\ITER_03XLS10.xls

04
D:\PCCS\00_RawData\01_CSV\CPV2010\04\ITER_04XLS10\ITER_04XLS10.xls

05
D:\PCCS\00_RawData\01_CSV\CPV2010\05\ITER_05XLS10\ITER_05XLS10.xls

06
D:\PCCS\00_RawData\01_CSV\CPV2010\06\ITER_06XLS10\ITER_06XLS10.xls

07
D:\PCCS\00_RawData\01_CSV\CPV2010\07\ITER_07XLS10\ITER_07XLS10.xls

08
D:\PCCS\00_RawData\01_CSV\CPV2010\08\ITER_08XLS10\ITER_08XLS10.xls

09
D:\PCCS\00_RawData\01_CSV\CPV2010\09\ITER_09XLS10\ITER_09XLS10.xls

10
D:\PCCS\00_RawData\01_CSV\CPV2010\10\ITER_10XLS10\ITER_10XLS10.xls

11
D:\PCCS\00_RawData\01_CSV\CPV2010\11\ITER_11XLS10\ITER_11XLS10.xls

12
D:\PCCS\00_RawData\01_CSV\CPV2010\12\ITER_12XLS10\ITER_12XLS10.xls

13
D:\PCCS\00_RawData\01_CSV\CPV2010\13\ITER_13XLS10\ITER_13XLS10.xls

14
D:\PCCS\00_RawData\01_CSV\CPV2010\14\ITER_14XLS10\ITER_14XLS10.xls

15
D:\

## 3. Estandarizacion del dataset

In [9]:
# Funcion para extraer datos de hoja tipo
# La funcion espera los siguientes valores:
# --- entidad: [str] clave geoestadistica de entidad de 2 digitos
# --- ruta: [str] ruta al archivo de excel que contiene la información

def cargahoja(ruta):
    # Abre el archivo de excel
    raw_data = pd.read_excel(ruta, sheetname='Hoja1', dtype={
        'ENTIDAD':'str',
        'MUN':'str',
        'LOC':'str',
    }).dropna()

    # Crea la columna CVE_MUN 
    raw_data['CVE_MUN'] = raw_data['ENTIDAD'].map(str) + raw_data['MUN']

    # Excluye los renglones que tienen infomracion de resumen
    raw_data = raw_data[raw_data['MUN'] != '000']
    
    # Borra columnas con informacion irrelevante o duplicada
    del (raw_data['ENTIDAD'])
    del (raw_data['MUN'])
    del (raw_data['NOM_ENT'])
    raw_data.set_index('CVE_MUN', inplace=True)
    
    return raw_data

Correr funcion sobre todos los archivos para importar los datos

In [10]:
dataframes = {}
for CVE_EDO, ruta in xlsfiles.items():
    print('procesando {} desde {}'.format(CVE_EDO, ruta))
    dataframes[CVE_EDO] = cargahoja(ruta)

procesando 01 desde D:\PCCS\00_RawData\01_CSV\CPV2010\01\ITER_01XLS10\ITER_01XLS10.xls
procesando 02 desde D:\PCCS\00_RawData\01_CSV\CPV2010\02\ITER_02XLS10\ITER_02XLS10.xls
procesando 03 desde D:\PCCS\00_RawData\01_CSV\CPV2010\03\ITER_03XLS10\ITER_03XLS10.xls
procesando 04 desde D:\PCCS\00_RawData\01_CSV\CPV2010\04\ITER_04XLS10\ITER_04XLS10.xls
procesando 05 desde D:\PCCS\00_RawData\01_CSV\CPV2010\05\ITER_05XLS10\ITER_05XLS10.xls
procesando 06 desde D:\PCCS\00_RawData\01_CSV\CPV2010\06\ITER_06XLS10\ITER_06XLS10.xls
procesando 07 desde D:\PCCS\00_RawData\01_CSV\CPV2010\07\ITER_07XLS10\ITER_07XLS10.xls
procesando 08 desde D:\PCCS\00_RawData\01_CSV\CPV2010\08\ITER_08XLS10\ITER_08XLS10.xls
procesando 09 desde D:\PCCS\00_RawData\01_CSV\CPV2010\09\ITER_09XLS10\ITER_09XLS10.xls
procesando 10 desde D:\PCCS\00_RawData\01_CSV\CPV2010\10\ITER_10XLS10\ITER_10XLS10.xls
procesando 11 desde D:\PCCS\00_RawData\01_CSV\CPV2010\11\ITER_11XLS10\ITER_11XLS10.xls
procesando 12 desde D:\PCCS\00_RawData\01_C

Los datos fueron guardados como diccionario de Python, es necesario convertirlos en un DataFrame unico antes de hacer la limpieza final.

In [11]:
# Unificacion de datos a un solo dataset
censo2010 = pd.DataFrame()
for CVE_EDO, dataframe in dataframes.items():
    censo2010 = censo2010.append(dataframe)

El dataset tiene 192,247 registros para 2456 municipios y 197 variables.

In [12]:
print('Registros: {}'.format(len(censo2010)))
print('Municipios: {}'.format(len(set(censo2010.index))))
print('Variables: {}'.format(len(list(censo2010))))

Registros: 192247
Municipios: 2456
Variables: 197


Es conveniente guardar el dataset en este estado como futura referencia. Los datos se encuentran desagregados a nivel LOCALIDAD, lo cual hace de este un archivo muy pesado. Además, por el momento solo se requiere el numero de viviendas incluido en la variable VIVTOT, por lo que después de guardar el dataset censo2010 se guardará un dataset con el numero total de viviendas a nivel MUNICIPAL.

### Guardado de datasets estándar

In [13]:
metadatos = {
    'Nombre del Dataset': 'Censo de Población y Vivienda 2010',
    'Descripcion del dataset': None,
    'Disponibilidad Temporal': '2010',
    'Periodo de actualizacion': 'Decada',
    'Nivel de Desagregacion': 'Localidad',
    'Notas': 's/n',
    'Fuente': 'INEGI (Microdatos)',
    'URL_Fuente': 'http://www.beta.inegi.org.mx/proyectos/ccpv/2010/?#section',
    'Dataset base': None,
}

Variables = {
    'CVE_MUN' : 'Clave geoestadística municipal de 5 dígitos compuesta por los dos digitos de ENTIDAD y los tres digitos de MUN',
    'ENTIDAD' : 'Clave de entidad federativa: Clave que identifica a la entidad federativa. ',
    'MUN' : 'Clave de municipio ó delegación: Clave que identifica al municipio al interior de una entidad federativa, conforme al marco geoestadístico nacional. ',
    'LOC' : 'Clave de localidad: Clave que identifica a la localidad, al interior de cada municipio (o delegación política) conforme al marco geoestadístico nacional. ',
    'AGEB' : 'Clave de ageb: Clave que identifica al ageb urbana, al interior de una localidad, de acuerdo a la desagregación del marco geoestadístico nacional.',
    'MZA' : 'Clave de manzana: Clave que identifica a la manzana, al interior de una ageb, de acuerdo a la desagregación del marco geoestadístico nacional.',
    'TIPO' : 'Tipo de ageb: Identifica si el área pertenece a un ageb urbano (u) o rural (r).',
    'POBTOT' : 'Población total: Total de personas que residen habitualmente en el país, entidad federativa, municipio y localidad. Incluye la estimación del número de personas en viviendas particulares sin información de ocupantes. Incluye a la población que no especificó su edad.',
    'POBMAS' : 'Población masculina: Total de hombres que residen habitualmente en el país, entidad federativa, municipio y localidad. Incluye la estimación del número de hombres en viviendas particulares sin información de ocupantes. Incluye a la población que no especificó su edad.',
    'POBFEM' : 'Población femenina: Total de mujeres que residen habitualmente en el país, entidad federativa, municipio y localidad. Incluye la estimación del número de mujeres en viviendas particulares sin información de ocupantes. Incluye a la población que no especificó su edad.',
    'P_0A2' : 'Población de 0 a 2 años : Personas de 0 a 2 años de edad.',
    'P_0A2_M' : 'Población masculina de 0 a 2 años: Hombres de 0 a 2 años de edad.',
    'P_0A2_F' : 'Población femenina de 0 a 2 años: Mujeres de 0 a 2 años de edad.',
    'P_3YMAS' : 'Población de  3 años y más: Personas de 3 a 130 años de edad.',
    'P_3YMAS_M' : 'Población masculina de 3 años y más: Hombres de 3 a 130 años de edad.',
    'P_3YMAS_F' : 'Población femenina de 3 años y más: Mujeres de 3 a 130 años de edad.',
    'P_5YMAS  ' : 'Población de  5 años y más: Personas de 5 a 130 años de edad.',
    'P_5YMAS_M' : 'Población masculina de 5 años y más: Hombres de 5 a 130 años de edad.',
    'P_5YMAS_F' : 'Población femenina de 5 años y más: Mujeres de 5 a 130 años de edad.',
    'P_12YMAS' : 'Población de 12 años y más: Personas de 12 a 130 años de edad.',
    'P_12YMAS_M' : 'Población masculina de 12 años y más: Hombres de 12 a 130 años de edad.',
    'P_12YMAS_F' : 'Población femenina de 12 años y más: Mujeres de 12 a 130 años de edad.',
    'P_15YMAS' : 'Población de 15 años y más: Personas de 15 a 130 años de edad.',
    'P_15YMAS_M' : 'Población masculina de 15 años y más: Hombres de 15 a 130 años de edad.',
    'P_15YMAS_F' : 'Población femenina de 15 años y más: Mujeres de 15 a 130 años de edad.',
    'P_18YMAS' : 'Población de 18 años y más: Personas de 18 a 130 años de edad.',
    'P_18YMAS_M' : 'Población masculina de 18 años y más: Hombres de 18 a 130 años de edad.',
    'P_18YMAS_F' : 'Población femenina de 18 años y más: Mujeres de 18 a 130 años de edad.',
    'P_3A5' : 'Población de 3 a 5 años: Personas de 3 a 5 años de edad.',
    'P_3A5_M' : 'Población masculina de 3 a 5 años: Hombres de 3 a 5 años de edad.',
    'P_3A5_F' : 'Población femenina de 3 a 5 años: Mujeres de 3 a 5 años de edad.',
    'P_6A11' : 'Población de 6 a 11 años: Personas de 6 a 11 años de edad.',
    'P_6A11_M' : 'Población masculina de 6 a 11 años: Hombres de 6 a 11 años de edad.',
    'P_6A11_F' : 'Población femenina de 6 a 11 años: Mujeres de 6 a 11 años de edad.',
    'P_8A14' : 'Población de 8 a 14 años: Personas de 8 a 14 años de edad.',
    'P_8A14_M' : 'Población masculina de 8 a 14 años: Hombres de 8 a 14 años de edad.',
    'P_8A14_F' : 'Población femenina de 8 a 14 años: Mujeres de 8 a 14 años de edad.',
    'P_12A14' : 'Población de 12 a 14 años: Personas de 12 a 14 años de edad.',
    'P_12A14_M' : 'Población masculina de 12 a 14 años: Hombres de 12 a 14 años de edad.',
    'P_12A14_F' : 'Población femenina de 12 a 14 años: Mujeres de 12 a 14 años de edad.',
    'P_15A17' : 'Población de 15 a 17 años: Personas de 15 a 17 años de edad.',
    'P_15A17_M' : 'Población masculina de 15 a 17 años: Hombres de 15 a 17 años de edad.',
    'P_15A17_F' : 'Población femenina de 15 a 17 años: Mujeres de 15 a 17 años de edad.',
    'P_18A24' : 'Población de 18 a 24 años: Personas de 18 a 24 años de edad.',
    'P_18A24_M' : 'Población masculina de 18 a 24 años: Hombres de 18 a 24 años de edad.',
    'P_18A24_F' : 'Población femenina de 18 a 24 años: Mujeres de 18 a 24 años de edad.',
    'P_15A49_F' : 'Población femenina de 15 a 49 años : Mujeres de 15 a 49 años de edad.',
    'P_60YMAS' : 'Población de 60 años y más: Personas de 60 a 130 años de edad.',
    'P_60YMAS_M' : 'Población masculina de 60 años y más: Hombres de 60 a 130 años de edad.',
    'P_60YMAS_F' : 'Población femenina de 60 años y más: Mujeres de 60 a 130 años de edad.',
    'REL_H_M' : 'Relación hombres-mujeres: Resultado de dividir el total de hombres entre el total de mujeres y multiplicarlo por cien. Indica el número de hombres por cada 100 mujeres. ',
    'POB0_14' : 'Población de cero a 14 años: Personas de cero a 14 años de edad.',
    'POB15_64' : 'Población de 15 a 64 años: Personas de 15 a 64 años de edad.',
    'POB65_MAS' : 'Población de 65 años y más: Personas de 65 a 130 años de edad.',
    'PROM_HNV' : 'Promedio de hijos nacidos vivos: Resultado de dividir el total de hijos nacidos vivos de las mujeres de 12 a 130 años de edad, entre el total de mujeres del mismo grupo de edad. Excluye a las mujeres que no especificaron el número de hijos.',
    'PNACENT' : 'Población nacida en la entidad: Personas nacidas en la misma entidad federativa.',
    'PNACENT_M' : 'Población masculina nacida en la entidad: Hombres nacidos en la misma entidad federativa.',
    'PNACENT_F' : 'Población femenina nacida en la entidad: Mujeres nacidas en la misma entidad federativa.',
    'PNACOE' : 'Población nacida en otra entidad : Personas nacidas en otra entidad federativa.',
    'PNACOE_M' : 'Población masculina nacida en otra entidad : Hombres nacidos en otra entidad federativa.',
    'PNACOE_F' : 'Población femenina nacida en otra entidad : Mujeres nacidas en otra entidad federativa.',
    'PRES2005' : 'Población de 5 años y más residente en la entidad en junio de 2005: Personas de 5 a 130 años de edad que en los años 2005 y 2010 residían en la misma entidad federativa.',
    'PRES2005_M' : 'Población masculina de 5 años y más residente en la entidad en junio de 2005: Hombres de 5 a 130 años de edad que en los años 2005 y 2010 residían en la misma entidad federativa.',
    'PRES2005_F' : 'Población femenina de 5 años y más residente en la entidad en junio de 2005: Mujeres de 5 a 130 años de edad que en los años 2005 y 2010 residían en la misma entidad federativa.',
    'PRESOE05' : 'Población de 5 años y más residente en otra entidad en  junio de 2005: Personas de 5 a 130 años de edad que en el año 2005 residían en otra entidad federativa.',
    'PRESOE05_M' : 'Población masculina de 5 años y más residente en otra entidad en  junio de 2005: Hombres de 5 a 130 años de edad que en el año 2005 residían en otra entidad federativa.',
    'PRESOE05_F' : 'Población femenina de 5 años y más residente en otra entidad en  junio de 2005: Mujeres de 5 a 130 años de edad que en el año 2005 residían en otra entidad federativa.',
    'P3YM_HLI' : 'Población de 3 años y más que habla alguna lengua indígena: Personas de 3 a 130 años de edad que hablan alguna lengua indígena.',
    'P3YM_HLI_M' : 'Población masculina de 3 años y más que habla alguna lengua indígena: Hombres de 3 a 130 años de edad que hablan alguna lengua indígena.',
    'P3YM_HLI_F' : 'Población femenina de 3 años y más que habla alguna lengua indígena: Mujeres de 3 a 130 años de edad que hablan alguna lengua indígena.',
    'P3HLINHE' : 'Población de 3 años y más que habla alguna lengua indígena y no habla español: Personas de 3 a 130 años de edad que hablan alguna lengua indígena y además no hablan español.',
    'P3HLINHE_M' : 'Población masculina de 3 años y más que habla alguna lengua indígena y no habla español: Hombres de 3 a 130 años de edad que hablan alguna lengua indígena y además no hablan español.',
    'P3HLINHE_F' : 'Población femenina de 3 años y más que habla alguna lengua indígena y no habla español: Mujeres de 3 a 130 años de edad que hablan alguna lengua indígena y además no hablan español.',
    'P3HLI_HE' : 'Población de 3 años y más que habla alguna lengua indígena y habla español: Personas de 3 a 130 años de edad que hablan alguna lengua indígena y además hablan español.',
    'P3HLI_HE_M' : 'Población masculina de 3 años y más que habla alguna lengua indígena y habla español: Hombres de 3 a 130 años de edad que hablan alguna lengua indígena y además hablan español.',
    'P3HLI_HE_F' : 'Población femenina de 3 años y más que habla alguna lengua indígena y habla español: Mujeres de 3 a 130 años de edad que hablan alguna lengua indígena y además hablan español.',
    'P5_HLI' : 'Población de 5 años y más que habla alguna lengua indígena: Personas de 5 años y más que hablan alguna lengua indígena.',
    'P5_HLI_NHE' : 'Población de 5 años y más que habla alguna lengua indígena y no habla español: Personas de 5 años y más que hablan alguna lengua indígena y no hablan español.',
    'P5_HLI_HE' : 'Población de 5 años y más que habla alguna lengua indígena y habla español: Personas de 5 años y más que hablan alguna lengua indígena y que hablan español.',
    'PHOG_IND' : 'Población en hogares censales indígenas: Total de personas que forman hogares censales donde el jefe del hogar o su cónyuge hablan alguna lengua indígena.',
    'PCON_LIM' : 'Población con limitación en la actividad: Personas que tienen dificultad para el desempeño y/o realización de tareas en la vida cotidiana.',
    'PCLIM_MOT' : 'Población con limitación para caminar o moverse, subir o bajar: Personas con dificultad para caminar o moverse, subir o bajar.',
    'PCLIM_VIS' : 'Población con limitación para ver, aún usando lentes: Personas con dificultad para ver, aún cuando usen lentes.',
    'PCLIM_LENG' : 'Población con limitación para hablar, comunicarse o conversar: Personas con dificultad para comunicarse con los demás o que tienen limitaciones para la recepción y producción de mensajes para hacerse entender a través del lenguaje, signos y símbolos.',
    'PCLIM_AUD' : 'Población con limitación para escuchar: Personas con dificultad para escuchar, aún usando aparato auditivo. ',
    'PCLIM_MOT2' : 'Población con limitación para vestirse, bañarse o comer: Personas con dificultad para bañarse, vestirse y/o comer.',
    'PCLIM_MEN' : 'Población con limitación para poner atención o aprender cosas sencillas: Personas con dificultad para mantener un nivel de atención en cosas sencillas. ',
    'PCLIM_MEN2' : 'Población con limitación mental: Personas con dificultad o con alguna limitación mental. ',
    'PSIN_LIM' : 'Población sin limitación en la actividad: Personas que no tienen dificultad para el desempeño y/o realización de tareas en la vida cotidiana.',
    'P3A5_NOA' : 'Población de 3 a 5 años que no asiste a la escuela : Personas de 3 a 5 años de edad que no van a la escuela.',
    'P3A5_NOA_M' : 'Población masculina de 3 a 5 años que no asiste a la escuela : Hombres de 3 a 5 años de edad que no van a la escuela.',
    'P3A5_NOA_F' : 'Población femenina de 3 a 5 años que no asiste a la escuela : Mujeres de 3 a 5 años de edad que no van a la escuela.',
    'P6A11_NOA' : 'Población de 6 a 11 años que no asiste a la escuela: Personas de 6 a 11 años de edad que no van a la escuela.',
    'P6A11_NOAM' : 'Población masculina de 6 a 11 años que no asiste a la escuela: Hombres de 6 a 11 años de edad que no van a la escuela.',
    'P6A11_NOAF' : 'Población femenina de 6 a 11 años que no asiste a la escuela: Mujeres de 6 a 11 años de edad que no van a la escuela.',
    'P12A14NOA' : 'Población de 12 a 14 años que no asiste a la escuela: Personas de 12 a 14 años de edad que no van a la escuela.',
    'P12A14NOAM' : 'Población masculina de 12 a 14 años que no asiste a la escuela: Hombres de 12 a 14 años de edad que no van a la escuela.',
    'P12A14NOAF' : 'Población femenina de 12 a 14 años que no asiste a la escuela: Mujeres de 12 a 14 años de edad que no van a la escuela.',
    'P15A17A ' : 'Población de 15 a 17 años que asiste a la escuela: Personas de 15 a 17 años de edad que van a la escuela.',
    'P15A17A_M' : 'Población masculina de 15 a 17 años que asiste a la escuela: Hombres de 15 a 17 años de edad que van a la escuela.',
    'P15A17A_F' : 'Población femenina de 15 a 17 años que asiste a la escuela: Mujeres de 15 a 17 años de edad que van a la escuela.',
    'P18A24A ' : 'Población de 18 a 24 años que asiste a la escuela: Personas de 18 a 24 años de edad que van a la escuela.',
    'P18A24A_M' : 'Población masculina de 18 a 24 años que asiste a la escuela: Hombres de 18 a 24 años de edad que van a la escuela.',
    'P18A24A_F' : 'Población femenina de 18 a 24 años que asiste a la escuela: Mujeres de 18 a 24 años de edad que van a la escuela.',
    'P8A14AN' : 'Población de 8 a 14 años que no saben leer y escribir: Personas de 8 a 14 años de edad que no saben leer y escribir.',
    'P8A14AN_M' : 'Población masculina de 8 a 14 años que no saben leer y escribir: Hombres de 8 a 14 años de edad que no saben leer y escribir.',
    'P8A14AN_F' : 'Población femenina  de 8 a 14 años que no saben leer y escribir: Mujeres de 8 a 14 años de edad que no saben leer y escribir.',
    'P15YM_AN' : 'Población de 15 años y más analfabeta: Personas de 15 a 130 años de edad que no saben leer ni escribir.',
    'P15YM_AN_M' : 'Población masculina de 15 años y más analfabeta: Hombres de 15 a 130 años de edad que no saben leer ni escribir.',
    'P15YM_AN_F' : 'Población femenina de 15 años y más analfabeta: Mujeres de 15 a 130 años de edad que no saben leer ni escribir.',
    'P15YM_SE' : 'Población de 15 años y más sin escolaridad: Personas de 15 a 130 años de edad que no aprobaron ningún grado de escolaridad o que solo tienen nivel preescolar ',
    'P15YM_SE_M' : 'Población masculina de 15 años y más sin escolaridad: Hombres de 15 a 130 años de edad que no aprobaron ningún grado de escolaridad o que solo tienen nivel preescolar',
    'P15YM_SE_F' : 'Población femenina de 15 años y más sin escolaridad: Mujeres de 15 a 130 años de edad que no aprobaron ningún grado de escolaridad o que solo tienen nivel preescolar',
    'P15PRI_IN' : 'Población de 15 años y más con primaria incompleta: Personas de 15 a 130 años de edad que tienen como máxima escolaridad hasta el quinto grado aprobado en primaria. Incluye a las personas que no especificaron los grados aprobados en el nivel señalado.',
    'P15PRI_INM' : 'Población masculina de 15 años y más con primaria incompleta: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad hasta el quinto grado aprobado en primaria. Incluye a los hombres que no especificaron los grados aprobados en el nivel señalado.',
    'P15PRI_INF' : 'Población femenina de 15 años y más con primaria incompleta: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad hasta el quinto grado aprobado en primaria. Incluye a las mujeres que no especificaron los grados aprobados en el nivel señalado.',
    'P15PRI_CO' : 'Población de 15 años y más con primaria completa: Personas de 15 a 130 años de edad que tienen como máxima escolaridad 6 grados aprobados en primaria. ',
    'P15PRI_COM' : 'Población masculina de 15 años y más con primaria completa: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad 6 grados aprobados en primaria.',
    'P15PRI_COF' : 'Población femenina de 15 años y más con primaria completa: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad 6 grados aprobados en primaria.',
    'P15SEC_IN' : 'Población de 15 años y más con secundaria incompleta: Personas de 15 a 130 años de edad que tienen como máxima escolaridad hasta segundo grado aprobado de secundaria. Incluye a las personas que no especificaron los grados aprobados en el nivel señalado.',
    'P15SEC_INM' : 'Población masculina de 15 años y más con secundaria incompleta: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad hasta segundo grado aprobado de secundaria. Incluye a los hombres que no especificaron los grados aprobados en el nivel señalado.',
    'P15SEC_INF' : 'Población femenina de 15 años y más con secundaria incompleta: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad hasta segundo grado aprobado de secundaria. Incluye a las mujeres que no especificaron los grados aprobados en el nivel señalado.',
    'P15SEC_CO' : 'Población de 15 años y más con secundaria completa: Personas de 15 a 130 años de edad que tienen como máxima escolaridad 3 grados aprobados en secundaria.',
    'P15SEC_COM' : 'Población masculina de 15 años y más con secundaria completa: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad 3 grados aprobados en secundaria.',
    'P15SEC_COF' : 'Población femenina de 15 años y más con secundaria completa: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad 3 grados aprobados en secundaria.',
    'P18YM_PB' : 'Población de 18 años y más con educación pos-básica: Personas de 18 a 130 años de edad que tienen como máxima escolaridad algún grado aprobado en: preparatoria ó bachillerato; normal básica, estudios técnicos o comerciales   con secundaria terminada; estudios técnicos o comerciales con preparatoria terminada; normal de licenciatura; licenciatura o profesional; maestría o doctorado. Incluye a las personas que no especificaron los grados aprobados en los niveles señalados.',
    'P18YM_PB_M' : 'Población masculina de 18 años y más con educación pos-básica: Hombres de 18 a 130 años de edad que tienen como máxima escolaridad algún grado aprobado en: preparatoria ó bachillerato; normal básica, estudios técnicos o comerciales   con secundaria terminada; estudios técnicos o comerciales con preparatoria terminada; normal de licenciatura; licenciatura o profesional; maestría o doctorado. Incluye a las personas que no especificaron los grados aprobados en los niveles señalados.',
    'P18YM_PB_F' : 'Población femenina de 18 años y más con educación pos-básica: Mujeres de 18 a 130 años de edad que tienen como máxima escolaridad algún grado aprobado en: preparatoria ó bachillerato; normal básica, estudios técnicos o comerciales   con secundaria terminada; estudios técnicos o comerciales con preparatoria terminada; normal de licenciatura; licenciatura o profesional; maestría o doctorado. Incluye a las personas que no especificaron los grados aprobados en los niveles señalados.',
    'GRAPROES' : 'Grado promedio de escolaridad: Resultado de dividir el monto de grados escolares aprobados por las personas de 15 a 130 años de edad entre las personas del mismo grupo de edad. Excluye a las personas que no especificaron los grados aprobados.',
    'GRAPROES_M ' : 'Grado promedio de escolaridad de la población masculina: Resultado de dividir el monto de grados escolares aprobados por los hombres de 15 a 130 años de edad entre los hombres del mismo grupo de edad. Excluye a los hombres que no especificaron los grados aprobados.',
    'GRAPROES_F' : 'Grado promedio de escolaridad de la población femenina: Resultado de dividir el monto de grados escolares aprobados por las mujeres de 15 a 130 años de edad entre las mujeres del mismo grupo de edad. Excluye a las mujeres que no especificaron los grados aprobados.',
    'PEA' : 'Población económicamente activa: Personas de 12 años y más que trabajaron; tenían trabajo pero no trabajaron o; buscaron trabajo en la semana de referencia.',
    'PEA_M' : 'Población masculina económicamente activa: Hombres de 12 años y más que trabajaron; tenían trabajo pero no trabajaron o; buscaron trabajo en la semana de referencia.',
    'PEA_F' : 'Población femenina económicamente activa: Mujeres de 12 años y más que trabajaron; tenían trabajo pero no trabajaron o; buscaron trabajo en la semana de referencia.',
    'PE_INAC' : 'Población no económicamente activa: Personas de 12 años y más pensionadas o jubiladas, estudiantes, dedicadas a los quehaceres del hogar, que tienen alguna limitación física o mental permanente que le impide trabajar. ',
    'PE_INAC_M' : 'Población masculina no económicamente activa: Hombres de 12 años y más pensionados o jubilados, estudiantes, dedicados a los quehaceres del hogar, que tienen alguna limitación física o mental permanente que le impide trabajar. ',
    'PE_INAC_F' : 'Población femenina no económicamente activa: Mujeres de 12 años y más pensionadas o jubiladas, estudiantes, dedicadas a los quehaceres del hogar, que tienen alguna limitación física o mental permanente que le impide trabajar. ',
    'POCUPADA' : 'Población ocupada: Personas de 12 a 130 años de edad que trabajaron o que no trabajaron pero sí tenían trabajo en la semana de referencia.',
    'POCUPADA_M' : 'Población masculina ocupada: Hombres de 12 a 130 años de edad que trabajaron o que no trabajaron pero sí tenían trabajo en la semana de referencia.',
    'POCUPADA_F' : 'Población femenina ocupada: Mujeres de 12 a 130 años de edad que trabajaron o que no trabajaron pero sí tenían trabajo en la semana de referencia.',
    'PDESOCUP' : 'Población desocupada: Personas de 12 a 130 años de edad que no tenían trabajo, pero buscaron trabajo en la semana de referencia. ',
    'PDESOCUP_M' : 'Población masculina desocupada: Hombres de 12 a 130 años de edad que no tenían trabajo, pero buscaron trabajo en la semana de referencia.',
    'PDESOCUP_F' : 'Población femenina desocupada: Mujeres de 12 a 130 años de edad que no tenían trabajo, pero buscaron trabajo en la semana de referencia.',
    'PSINDER' : 'Población sin derechohabiencia a servicios de salud  : Total de personas que no tienen derecho a recibir servicios médicos en ninguna institución pública o privada.',
    'PDER_SS ' : 'Población derechohabiente a servicios de salud: Total de personas que tienen derecho a recibir servicios médicos en alguna institución de salud pública o privada como: el instituto mexicano del seguro social (imss), el instituto de seguridad y servicios sociales de los trabajadores del estado (issste e issste estatal), petróleos mexicanos (pemex), la secretaría de la defensa nacional (sedena), la secretaría de marina armada de méxico (semar), el sistema de protección social en salud o en otra.',
    'PDER_IMSS ' : 'Población derechohabiente del imss  : Total de personas que tienen derecho a recibir servicios médicos en el instituto mexicano del seguro social (imss).',
    'PDER_ISTE' : 'Población derechohabiente del issste: Total de personas que tienen derecho a recibir servicios médicos en el instituto de seguridad y servicios sociales de los trabajadores del estado.',
    'PDER_ISTEE' : 'Población derechohabiente del issste estatal: Total de personas que tienen derecho a recibir servicios médicos en los institutos de seguridad social de los estados (issset, isssemym, issstezac, issspea o isssteson).',
    'PDER_SEGP' : 'Población derechohabiente del seguro popular o seguro médico para una nueva generación: Total de personas que tienen derecho a recibir servicios médicos en la secretaría de salud, mediante el sistema de protección en salud (seguro popular).',
    'P12YM_SOLT' : 'Población soltera o nunca unida de 12 años y más: Personas solteras de 12 a 130 años de edad. ',
    'P12YM_CASA' : 'Población casada o unida de 12 años y más : Personas de 12 a 130 años de edad que viven con su pareja en unión libre; casadas solo por el civil; casadas solo religiosamente o; casadas por el civil y religiosamente. ',
    'P12YM_SEPA' : 'Población que estuvo casada o unida de 12 años y más : Personas de 12 a 130 años de edad que están separadas, divorciadas o viudas. ',
    'PCATOLICA' : 'Población con religión católica : Personas con religión católica. ',
    'PNCATOLICA' : 'Protestantes, evangélicas y bíblicas diferentes de evangélicas: Personas con religiones protestantes históricas, pentecostales, neopentecostales, iglesia del dios vivo, columna y apoyo de la verdad, la luz del mundo, cristianas,  evangélicas y bíblicas diferentes de las evangélicas.',
    'POTRAS_REL' : 'Población con otras religiones diferentes a las anteriores: Personas con religiones de origen oriental, judaico, islámico, new age, escuelas esotéricas, raíces étnicas, espiritualistas, ortodoxos,  otros movimientos religiosos y cultos populares.',
    'PSIN_RELIG' : 'Población sin religión: Personas sin adscripción religiosa. Incluye ateísmo. ',
    'TOTHOG' : 'Total de hogares censales: Hogares en viviendas particulares habitadas. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada. ',
    'HOGJEF_M' : 'Hogares censales con jefatura masculina: Hogares en viviendas particulares habitadas donde el jefe es hombre. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'HOGJEF_F' : 'Hogares censales con jefatura femenina: Hogares en viviendas particulares habitadas donde el jefe es mujer. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'POBHOG' : 'Población en hogares censales: Personas en hogares censales. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'PHOGJEF_M' : 'Población en hogares censales con jefatura masculina: Personas en hogares censales donde el jefe es hombre. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'PHOGJEF_F' : 'Población en hogares censales con jefatura femenina: Personas en hogares censales donde el jefe es mujer. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'VIVTOT' : 'Total de viviendas: Viviendas particulares habitadas, deshabitadas, de uso temporal y colectivas. Incluye a las viviendas particulares sin información de sus ocupantes.',
    'TVIVHAB' : 'Total de viviendas habitadas: Viviendas particulares y colectivas habitadas. Incluye a las viviendas particulares sin información de sus ocupantes.',
    'TVIVPAR' : 'Total de viviendas particulares: Viviendas particulares habitadas, deshabitadas y de uso temporal. Excluye a las viviendas particulares sin información de ocupantes.',
    'VIVPAR_HAB' : 'Viviendas particulares habitadas: Viviendas particulares habitadas de cualquier clase: casa independiente, departamento en edificio, vivienda o cuarto en vecindad, vivienda o cuarto de azotea, local no construido para habitación, vivienda móvil, refugios o clase no especificada. Excluye a las viviendas particulares sin información de ocupantes.',
    'TVIVPARHAB' : 'Total de viviendas particulares habitadas: Viviendas particulares habitadas de cualquier clase: casa independiente, departamento en edificio, vivienda o cuarto en vecindad, vivienda o cuarto de azotea, local no construido para habitación, vivienda móvil, refugios o clase no especificada Incluye a las viviendas particulares sin información de ocupantes.',
    'VIVPAR_DES' : 'Viviendas particulares deshabitadas: Viviendas particulares deshabitadas.',
    'VIVPAR_UT' : 'Viviendas particulares de uso temporal: Viviendas particulares de uso temporal.',
    'OCUPVIVPAR' : 'Ocupantes en viviendas particulares habitadas: Personas que residen en viviendas particulares habitadas de cualquier clase: casa independiente, departamento en edificio, vivienda o cuarto en vecindad, vivienda o cuarto de azotea, local no construido para habitación, vivienda móvil, refugios o clase no especificada. Excluye la estimación del número de personas en viviendas particulares sin información de ocupantes.',
    'PROM_OCUP' : 'Promedio de ocupantes en viviendas particulares habitadas: Resultado de dividir el número de personas que residen en viviendas particulares habitadas, entre el número de esas viviendas. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'PRO_OCUP_C' : 'Promedio de ocupantes por cuarto en viviendas particulares habitadas: Resultado de dividir el número de personas que residen en viviendas particulares habitadas entre el número de cuartos de esas viviendas. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_PISODT' : 'Viviendas particulares habitadas con piso de material diferente de tierra: Viviendas particulares habitadas con piso de cemento o firme, madera, mosaico u otro material. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_PISOTI' : 'Viviendas particulares habitadas con piso de tierra: Viviendas particulares habitadas con piso de tierra. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_1DOR' : 'Viviendas particulares habitadas con un dormitorio: Viviendas particulares habitadas donde sólo uno de los cuartos se usa para dormir. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_2YMASD' : 'Viviendas particulares habitadas con dos dormitorios y más: Viviendas particulares habitadas que usan para dormir entre 2 y 25 cuartos. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_1CUART' : 'Viviendas particulares habitadas con un solo cuarto : Viviendas particulares habitadas que tienen un solo cuarto. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'VPH_2CUART' : 'Viviendas particulares habitadas con dos cuartos: Viviendas particulares habitadas que tienen dos cuartos. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'VPH_3YMASC ' : 'Viviendas particulares habitadas con 3 cuartos y más: Viviendas particulares habitadas que tienen entre 3 y 25 cuartos. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_C_ELEC' : 'Viviendas particulares habitadas que disponen de luz eléctrica: Viviendas particulares habitadas que tienen luz eléctrica. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_S_ELEC' : 'Viviendas particulares habitadas que no disponen de luz eléctrica: Viviendas particulares habitadas que no tienen luz eléctrica. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_AGUADV' : 'Viviendas particulares habitadas que disponen de agua entubada en  el ámbito de la vivienda: Viviendas particulares habitadas que tienen disponibilidad de agua entubada dentro de la vivienda, o fuera de la vivienda pero dentro del terreno. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. ',
    'VPH_AGUAFV' : 'Viviendas particulares habitadas que no disponen de agua entubada en el ámbito de la vivienda: Viviendas particulares habitadas que tienen disponibilidad de agua de una llave pública o hidrante, de otra vivienda, de pipa, de pozo, río, arroyo, lago u otro. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_EXCSA' : 'Viviendas particulares habitadas que disponen de excusado o sanitario: Viviendas particulares habitadas que tienen excusado, retrete, sanitario, letrina u hoyo negro. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'VPH_DRENAJ' : 'Viviendas particulares habitadas que disponen de drenaje: Viviendas particulares habitadas que tienen drenaje conectado a la red pública, fosa séptica, barranca, grieta, río, lago o mar. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. ',
    'VPH_NODREN' : 'Viviendas particulares habitadas que no disponen de drenaje: Viviendas particulares habitadas que no tienen drenaje. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. ',
    'VPH_C_SERV' : 'Viviendas particulares habitadas que disponen de luz eléctrica, agua entubada de la red pública y drenaje : Viviendas particulares habitadas que tienen luz eléctrica, agua entubada dentro o fuera de la vivienda, pero dentro del terreno, así como drenaje. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_SNBIEN' : 'Viviendas particulares habitadas sin ningún bien: Viviendas particulares habitadas que no disponen de radio, televisión, refrigerador, lavadora, automóvil, computadora, teléfono fijo, celular ni internet. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_RADIO' : 'Viviendas particulares habitadas que disponen de radio: Viviendas particulares habitadas que tienen radio. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_TV' : 'Viviendas particulares habitadas que disponen de televisor: Viviendas particulares habitadas que tienen televisor. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_REFRI' : 'Viviendas particulares habitadas que disponen de refrigerador: Viviendas particulares habitadas que tienen refrigerador. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_LAVAD' : 'Viviendas particulares habitadas que disponen de lavadora: Viviendas particulares habitadas que tienen lavadora. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_AUTOM' : 'Viviendas particulares habitadas que disponen de automóvil o camioneta: Viviendas particulares habitadas que tienen automóvil o camioneta. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_PC ' : 'Viviendas particulares habitadas que disponen de computadora: Viviendas particulares habitadas que tienen computadora. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_TELEF' : 'Viviendas particulares habitadas que disponen de línea telefónica fija: Viviendas particulares habitadas que tienen línea telefónica fija. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_CEL' : 'Viviendas particulares habitadas que disponen de teléfono celular: Viviendas particulares habitadas que tienen teléfono celular. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_INTER' : 'Viviendas particulares habitadas que disponen de internet: Viviendas particulares habitadas que tienen servicio de internet. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'ENTIDAD' : 'Clave de entidad federativa: Clave que identifica a la entidad federativa. ',
    'MUN' : 'Clave de municipio ó delegación: Clave que identifica al municipio al interior de una entidad federativa, conforme al marco geoestadístico nacional. ',
    'LOC' : 'Clave de localidad: Clave que identifica a la localidad, al interior de cada municipio (o delegación política) conforme al marco geoestadístico nacional. ',
    'AGEB' : 'Clave de ageb: Clave que identifica al ageb urbana, al interior de una localidad, de acuerdo a la desagregación del marco geoestadístico nacional.',
    'MZA' : 'Clave de manzana: Clave que identifica a la manzana, al interior de una ageb, de acuerdo a la desagregación del marco geoestadístico nacional.',
    'TIPO' : 'Tipo de ageb: Identifica si el área pertenece a un ageb urbano (u) o rural (r).',
    'POBTOT' : 'Población total: Total de personas que residen habitualmente en el país, entidad federativa, municipio y localidad. Incluye la estimación del número de personas en viviendas particulares sin información de ocupantes. Incluye a la población que no especificó su edad.',
    'POBMAS' : 'Población masculina: Total de hombres que residen habitualmente en el país, entidad federativa, municipio y localidad. Incluye la estimación del número de hombres en viviendas particulares sin información de ocupantes. Incluye a la población que no especificó su edad.',
    'POBFEM' : 'Población femenina: Total de mujeres que residen habitualmente en el país, entidad federativa, municipio y localidad. Incluye la estimación del número de mujeres en viviendas particulares sin información de ocupantes. Incluye a la población que no especificó su edad.',
    'P_0A2' : 'Población de 0 a 2 años : Personas de 0 a 2 años de edad.',
    'P_0A2_M' : 'Población masculina de 0 a 2 años: Hombres de 0 a 2 años de edad.',
    'P_0A2_F' : 'Población femenina de 0 a 2 años: Mujeres de 0 a 2 años de edad.',
    'P_3YMAS' : 'Población de  3 años y más: Personas de 3 a 130 años de edad.',
    'P_3YMAS_M' : 'Población masculina de 3 años y más: Hombres de 3 a 130 años de edad.',
    'P_3YMAS_F' : 'Población femenina de 3 años y más: Mujeres de 3 a 130 años de edad.',
    'P_5YMAS  ' : 'Población de  5 años y más: Personas de 5 a 130 años de edad.',
    'P_5YMAS_M' : 'Población masculina de 5 años y más: Hombres de 5 a 130 años de edad.',
    'P_5YMAS_F' : 'Población femenina de 5 años y más: Mujeres de 5 a 130 años de edad.',
    'P_12YMAS' : 'Población de 12 años y más: Personas de 12 a 130 años de edad.',
    'P_12YMAS_M' : 'Población masculina de 12 años y más: Hombres de 12 a 130 años de edad.',
    'P_12YMAS_F' : 'Población femenina de 12 años y más: Mujeres de 12 a 130 años de edad.',
    'P_15YMAS' : 'Población de 15 años y más: Personas de 15 a 130 años de edad.',
    'P_15YMAS_M' : 'Población masculina de 15 años y más: Hombres de 15 a 130 años de edad.',
    'P_15YMAS_F' : 'Población femenina de 15 años y más: Mujeres de 15 a 130 años de edad.',
    'P_18YMAS' : 'Población de 18 años y más: Personas de 18 a 130 años de edad.',
    'P_18YMAS_M' : 'Población masculina de 18 años y más: Hombres de 18 a 130 años de edad.',
    'P_18YMAS_F' : 'Población femenina de 18 años y más: Mujeres de 18 a 130 años de edad.',
    'P_3A5' : 'Población de 3 a 5 años: Personas de 3 a 5 años de edad.',
    'P_3A5_M' : 'Población masculina de 3 a 5 años: Hombres de 3 a 5 años de edad.',
    'P_3A5_F' : 'Población femenina de 3 a 5 años: Mujeres de 3 a 5 años de edad.',
    'P_6A11' : 'Población de 6 a 11 años: Personas de 6 a 11 años de edad.',
    'P_6A11_M' : 'Población masculina de 6 a 11 años: Hombres de 6 a 11 años de edad.',
    'P_6A11_F' : 'Población femenina de 6 a 11 años: Mujeres de 6 a 11 años de edad.',
    'P_8A14' : 'Población de 8 a 14 años: Personas de 8 a 14 años de edad.',
    'P_8A14_M' : 'Población masculina de 8 a 14 años: Hombres de 8 a 14 años de edad.',
    'P_8A14_F' : 'Población femenina de 8 a 14 años: Mujeres de 8 a 14 años de edad.',
    'P_12A14' : 'Población de 12 a 14 años: Personas de 12 a 14 años de edad.',
    'P_12A14_M' : 'Población masculina de 12 a 14 años: Hombres de 12 a 14 años de edad.',
    'P_12A14_F' : 'Población femenina de 12 a 14 años: Mujeres de 12 a 14 años de edad.',
    'P_15A17' : 'Población de 15 a 17 años: Personas de 15 a 17 años de edad.',
    'P_15A17_M' : 'Población masculina de 15 a 17 años: Hombres de 15 a 17 años de edad.',
    'P_15A17_F' : 'Población femenina de 15 a 17 años: Mujeres de 15 a 17 años de edad.',
    'P_18A24' : 'Población de 18 a 24 años: Personas de 18 a 24 años de edad.',
    'P_18A24_M' : 'Población masculina de 18 a 24 años: Hombres de 18 a 24 años de edad.',
    'P_18A24_F' : 'Población femenina de 18 a 24 años: Mujeres de 18 a 24 años de edad.',
    'P_15A49_F' : 'Población femenina de 15 a 49 años : Mujeres de 15 a 49 años de edad.',
    'P_60YMAS' : 'Población de 60 años y más: Personas de 60 a 130 años de edad.',
    'P_60YMAS_M' : 'Población masculina de 60 años y más: Hombres de 60 a 130 años de edad.',
    'P_60YMAS_F' : 'Población femenina de 60 años y más: Mujeres de 60 a 130 años de edad.',
    'REL_H_M' : 'Relación hombres-mujeres: Resultado de dividir el total de hombres entre el total de mujeres y multiplicarlo por cien. Indica el número de hombres por cada 100 mujeres. ',
    'POB0_14' : 'Población de cero a 14 años: Personas de cero a 14 años de edad.',
    'POB15_64' : 'Población de 15 a 64 años: Personas de 15 a 64 años de edad.',
    'POB65_MAS' : 'Población de 65 años y más: Personas de 65 a 130 años de edad.',
    'PROM_HNV' : 'Promedio de hijos nacidos vivos: Resultado de dividir el total de hijos nacidos vivos de las mujeres de 12 a 130 años de edad, entre el total de mujeres del mismo grupo de edad. Excluye a las mujeres que no especificaron el número de hijos.',
    'PNACENT' : 'Población nacida en la entidad: Personas nacidas en la misma entidad federativa.',
    'PNACENT_M' : 'Población masculina nacida en la entidad: Hombres nacidos en la misma entidad federativa.',
    'PNACENT_F' : 'Población femenina nacida en la entidad: Mujeres nacidas en la misma entidad federativa.',
    'PNACOE' : 'Población nacida en otra entidad : Personas nacidas en otra entidad federativa.',
    'PNACOE_M' : 'Población masculina nacida en otra entidad : Hombres nacidos en otra entidad federativa.',
    'PNACOE_F' : 'Población femenina nacida en otra entidad : Mujeres nacidas en otra entidad federativa.',
    'PRES2005' : 'Población de 5 años y más residente en la entidad en junio de 2005: Personas de 5 a 130 años de edad que en los años 2005 y 2010 residían en la misma entidad federativa.',
    'PRES2005_M' : 'Población masculina de 5 años y más residente en la entidad en junio de 2005: Hombres de 5 a 130 años de edad que en los años 2005 y 2010 residían en la misma entidad federativa.',
    'PRES2005_F' : 'Población femenina de 5 años y más residente en la entidad en junio de 2005: Mujeres de 5 a 130 años de edad que en los años 2005 y 2010 residían en la misma entidad federativa.',
    'PRESOE05' : 'Población de 5 años y más residente en otra entidad en  junio de 2005: Personas de 5 a 130 años de edad que en el año 2005 residían en otra entidad federativa.',
    'PRESOE05_M' : 'Población masculina de 5 años y más residente en otra entidad en  junio de 2005: Hombres de 5 a 130 años de edad que en el año 2005 residían en otra entidad federativa.',
    'PRESOE05_F' : 'Población femenina de 5 años y más residente en otra entidad en  junio de 2005: Mujeres de 5 a 130 años de edad que en el año 2005 residían en otra entidad federativa.',
    'P3YM_HLI' : 'Población de 3 años y más que habla alguna lengua indígena: Personas de 3 a 130 años de edad que hablan alguna lengua indígena.',
    'P3YM_HLI_M' : 'Población masculina de 3 años y más que habla alguna lengua indígena: Hombres de 3 a 130 años de edad que hablan alguna lengua indígena.',
    'P3YM_HLI_F' : 'Población femenina de 3 años y más que habla alguna lengua indígena: Mujeres de 3 a 130 años de edad que hablan alguna lengua indígena.',
    'P3HLINHE' : 'Población de 3 años y más que habla alguna lengua indígena y no habla español: Personas de 3 a 130 años de edad que hablan alguna lengua indígena y además no hablan español.',
    'P3HLINHE_M' : 'Población masculina de 3 años y más que habla alguna lengua indígena y no habla español: Hombres de 3 a 130 años de edad que hablan alguna lengua indígena y además no hablan español.',
    'P3HLINHE_F' : 'Población femenina de 3 años y más que habla alguna lengua indígena y no habla español: Mujeres de 3 a 130 años de edad que hablan alguna lengua indígena y además no hablan español.',
    'P3HLI_HE' : 'Población de 3 años y más que habla alguna lengua indígena y habla español: Personas de 3 a 130 años de edad que hablan alguna lengua indígena y además hablan español.',
    'P3HLI_HE_M' : 'Población masculina de 3 años y más que habla alguna lengua indígena y habla español: Hombres de 3 a 130 años de edad que hablan alguna lengua indígena y además hablan español.',
    'P3HLI_HE_F' : 'Población femenina de 3 años y más que habla alguna lengua indígena y habla español: Mujeres de 3 a 130 años de edad que hablan alguna lengua indígena y además hablan español.',
    'P5_HLI' : 'Población de 5 años y más que habla alguna lengua indígena: Personas de 5 años y más que hablan alguna lengua indígena.',
    'P5_HLI_NHE' : 'Población de 5 años y más que habla alguna lengua indígena y no habla español: Personas de 5 años y más que hablan alguna lengua indígena y no hablan español.',
    'P5_HLI_HE' : 'Población de 5 años y más que habla alguna lengua indígena y habla español: Personas de 5 años y más que hablan alguna lengua indígena y que hablan español.',
    'PHOG_IND' : 'Población en hogares censales indígenas: Total de personas que forman hogares censales donde el jefe del hogar o su cónyuge hablan alguna lengua indígena.',
    'PCON_LIM' : 'Población con limitación en la actividad: Personas que tienen dificultad para el desempeño y/o realización de tareas en la vida cotidiana.',
    'PCLIM_MOT' : 'Población con limitación para caminar o moverse, subir o bajar: Personas con dificultad para caminar o moverse, subir o bajar.',
    'PCLIM_VIS' : 'Población con limitación para ver, aún usando lentes: Personas con dificultad para ver, aún cuando usen lentes.',
    'PCLIM_LENG' : 'Población con limitación para hablar, comunicarse o conversar: Personas con dificultad para comunicarse con los demás o que tienen limitaciones para la recepción y producción de mensajes para hacerse entender a través del lenguaje, signos y símbolos.',
    'PCLIM_AUD' : 'Población con limitación para escuchar: Personas con dificultad para escuchar, aún usando aparato auditivo. ',
    'PCLIM_MOT2' : 'Población con limitación para vestirse, bañarse o comer: Personas con dificultad para bañarse, vestirse y/o comer.',
    'PCLIM_MEN' : 'Población con limitación para poner atención o aprender cosas sencillas: Personas con dificultad para mantener un nivel de atención en cosas sencillas. ',
    'PCLIM_MEN2' : 'Población con limitación mental: Personas con dificultad o con alguna limitación mental. ',
    'PSIN_LIM' : 'Población sin limitación en la actividad: Personas que no tienen dificultad para el desempeño y/o realización de tareas en la vida cotidiana.',
    'P3A5_NOA' : 'Población de 3 a 5 años que no asiste a la escuela : Personas de 3 a 5 años de edad que no van a la escuela.',
    'P3A5_NOA_M' : 'Población masculina de 3 a 5 años que no asiste a la escuela : Hombres de 3 a 5 años de edad que no van a la escuela.',
    'P3A5_NOA_F' : 'Población femenina de 3 a 5 años que no asiste a la escuela : Mujeres de 3 a 5 años de edad que no van a la escuela.',
    'P6A11_NOA' : 'Población de 6 a 11 años que no asiste a la escuela: Personas de 6 a 11 años de edad que no van a la escuela.',
    'P6A11_NOAM' : 'Población masculina de 6 a 11 años que no asiste a la escuela: Hombres de 6 a 11 años de edad que no van a la escuela.',
    'P6A11_NOAF' : 'Población femenina de 6 a 11 años que no asiste a la escuela: Mujeres de 6 a 11 años de edad que no van a la escuela.',
    'P12A14NOA' : 'Población de 12 a 14 años que no asiste a la escuela: Personas de 12 a 14 años de edad que no van a la escuela.',
    'P12A14NOAM' : 'Población masculina de 12 a 14 años que no asiste a la escuela: Hombres de 12 a 14 años de edad que no van a la escuela.',
    'P12A14NOAF' : 'Población femenina de 12 a 14 años que no asiste a la escuela: Mujeres de 12 a 14 años de edad que no van a la escuela.',
    'P15A17A ' : 'Población de 15 a 17 años que asiste a la escuela: Personas de 15 a 17 años de edad que van a la escuela.',
    'P15A17A_M' : 'Población masculina de 15 a 17 años que asiste a la escuela: Hombres de 15 a 17 años de edad que van a la escuela.',
    'P15A17A_F' : 'Población femenina de 15 a 17 años que asiste a la escuela: Mujeres de 15 a 17 años de edad que van a la escuela.',
    'P18A24A ' : 'Población de 18 a 24 años que asiste a la escuela: Personas de 18 a 24 años de edad que van a la escuela.',
    'P18A24A_M' : 'Población masculina de 18 a 24 años que asiste a la escuela: Hombres de 18 a 24 años de edad que van a la escuela.',
    'P18A24A_F' : 'Población femenina de 18 a 24 años que asiste a la escuela: Mujeres de 18 a 24 años de edad que van a la escuela.',
    'P8A14AN' : 'Población de 8 a 14 años que no saben leer y escribir: Personas de 8 a 14 años de edad que no saben leer y escribir.',
    'P8A14AN_M' : 'Población masculina de 8 a 14 años que no saben leer y escribir: Hombres de 8 a 14 años de edad que no saben leer y escribir.',
    'P8A14AN_F' : 'Población femenina  de 8 a 14 años que no saben leer y escribir: Mujeres de 8 a 14 años de edad que no saben leer y escribir.',
    'P15YM_AN' : 'Población de 15 años y más analfabeta: Personas de 15 a 130 años de edad que no saben leer ni escribir.',
    'P15YM_AN_M' : 'Población masculina de 15 años y más analfabeta: Hombres de 15 a 130 años de edad que no saben leer ni escribir.',
    'P15YM_AN_F' : 'Población femenina de 15 años y más analfabeta: Mujeres de 15 a 130 años de edad que no saben leer ni escribir.',
    'P15YM_SE' : 'Población de 15 años y más sin escolaridad: Personas de 15 a 130 años de edad que no aprobaron ningún grado de escolaridad o que solo tienen nivel preescolar ',
    'P15YM_SE_M' : 'Población masculina de 15 años y más sin escolaridad: Hombres de 15 a 130 años de edad que no aprobaron ningún grado de escolaridad o que solo tienen nivel preescolar',
    'P15YM_SE_F' : 'Población femenina de 15 años y más sin escolaridad: Mujeres de 15 a 130 años de edad que no aprobaron ningún grado de escolaridad o que solo tienen nivel preescolar',
    'P15PRI_IN' : 'Población de 15 años y más con primaria incompleta: Personas de 15 a 130 años de edad que tienen como máxima escolaridad hasta el quinto grado aprobado en primaria. Incluye a las personas que no especificaron los grados aprobados en el nivel señalado.',
    'P15PRI_INM' : 'Población masculina de 15 años y más con primaria incompleta: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad hasta el quinto grado aprobado en primaria. Incluye a los hombres que no especificaron los grados aprobados en el nivel señalado.',
    'P15PRI_INF' : 'Población femenina de 15 años y más con primaria incompleta: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad hasta el quinto grado aprobado en primaria. Incluye a las mujeres que no especificaron los grados aprobados en el nivel señalado.',
    'P15PRI_CO' : 'Población de 15 años y más con primaria completa: Personas de 15 a 130 años de edad que tienen como máxima escolaridad 6 grados aprobados en primaria. ',
    'P15PRI_COM' : 'Población masculina de 15 años y más con primaria completa: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad 6 grados aprobados en primaria.',
    'P15PRI_COF' : 'Población femenina de 15 años y más con primaria completa: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad 6 grados aprobados en primaria.',
    'P15SEC_IN' : 'Población de 15 años y más con secundaria incompleta: Personas de 15 a 130 años de edad que tienen como máxima escolaridad hasta segundo grado aprobado de secundaria. Incluye a las personas que no especificaron los grados aprobados en el nivel señalado.',
    'P15SEC_INM' : 'Población masculina de 15 años y más con secundaria incompleta: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad hasta segundo grado aprobado de secundaria. Incluye a los hombres que no especificaron los grados aprobados en el nivel señalado.',
    'P15SEC_INF' : 'Población femenina de 15 años y más con secundaria incompleta: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad hasta segundo grado aprobado de secundaria. Incluye a las mujeres que no especificaron los grados aprobados en el nivel señalado.',
    'P15SEC_CO' : 'Población de 15 años y más con secundaria completa: Personas de 15 a 130 años de edad que tienen como máxima escolaridad 3 grados aprobados en secundaria.',
    'P15SEC_COM' : 'Población masculina de 15 años y más con secundaria completa: Hombres de 15 a 130 años de edad que tienen como máxima escolaridad 3 grados aprobados en secundaria.',
    'P15SEC_COF' : 'Población femenina de 15 años y más con secundaria completa: Mujeres de 15 a 130 años de edad que tienen como máxima escolaridad 3 grados aprobados en secundaria.',
    'P18YM_PB' : 'Población de 18 años y más con educación pos-básica: Personas de 18 a 130 años de edad que tienen como máxima escolaridad algún grado aprobado en: preparatoria ó bachillerato; normal básica, estudios técnicos o comerciales   con secundaria terminada; estudios técnicos o comerciales con preparatoria terminada; normal de licenciatura; licenciatura o profesional; maestría o doctorado. Incluye a las personas que no especificaron los grados aprobados en los niveles señalados.',
    'P18YM_PB_M' : 'Población masculina de 18 años y más con educación pos-básica: Hombres de 18 a 130 años de edad que tienen como máxima escolaridad algún grado aprobado en: preparatoria ó bachillerato; normal básica, estudios técnicos o comerciales   con secundaria terminada; estudios técnicos o comerciales con preparatoria terminada; normal de licenciatura; licenciatura o profesional; maestría o doctorado. Incluye a las personas que no especificaron los grados aprobados en los niveles señalados.',
    'P18YM_PB_F' : 'Población femenina de 18 años y más con educación pos-básica: Mujeres de 18 a 130 años de edad que tienen como máxima escolaridad algún grado aprobado en: preparatoria ó bachillerato; normal básica, estudios técnicos o comerciales   con secundaria terminada; estudios técnicos o comerciales con preparatoria terminada; normal de licenciatura; licenciatura o profesional; maestría o doctorado. Incluye a las personas que no especificaron los grados aprobados en los niveles señalados.',
    'GRAPROES' : 'Grado promedio de escolaridad: Resultado de dividir el monto de grados escolares aprobados por las personas de 15 a 130 años de edad entre las personas del mismo grupo de edad. Excluye a las personas que no especificaron los grados aprobados.',
    'GRAPROES_M ' : 'Grado promedio de escolaridad de la población masculina: Resultado de dividir el monto de grados escolares aprobados por los hombres de 15 a 130 años de edad entre los hombres del mismo grupo de edad. Excluye a los hombres que no especificaron los grados aprobados.',
    'GRAPROES_F' : 'Grado promedio de escolaridad de la población femenina: Resultado de dividir el monto de grados escolares aprobados por las mujeres de 15 a 130 años de edad entre las mujeres del mismo grupo de edad. Excluye a las mujeres que no especificaron los grados aprobados.',
    'PEA' : 'Población económicamente activa: Personas de 12 años y más que trabajaron; tenían trabajo pero no trabajaron o; buscaron trabajo en la semana de referencia.',
    'PEA_M' : 'Población masculina económicamente activa: Hombres de 12 años y más que trabajaron; tenían trabajo pero no trabajaron o; buscaron trabajo en la semana de referencia.',
    'PEA_F' : 'Población femenina económicamente activa: Mujeres de 12 años y más que trabajaron; tenían trabajo pero no trabajaron o; buscaron trabajo en la semana de referencia.',
    'PE_INAC' : 'Población no económicamente activa: Personas de 12 años y más pensionadas o jubiladas, estudiantes, dedicadas a los quehaceres del hogar, que tienen alguna limitación física o mental permanente que le impide trabajar. ',
    'PE_INAC_M' : 'Población masculina no económicamente activa: Hombres de 12 años y más pensionados o jubilados, estudiantes, dedicados a los quehaceres del hogar, que tienen alguna limitación física o mental permanente que le impide trabajar. ',
    'PE_INAC_F' : 'Población femenina no económicamente activa: Mujeres de 12 años y más pensionadas o jubiladas, estudiantes, dedicadas a los quehaceres del hogar, que tienen alguna limitación física o mental permanente que le impide trabajar. ',
    'POCUPADA' : 'Población ocupada: Personas de 12 a 130 años de edad que trabajaron o que no trabajaron pero sí tenían trabajo en la semana de referencia.',
    'POCUPADA_M' : 'Población masculina ocupada: Hombres de 12 a 130 años de edad que trabajaron o que no trabajaron pero sí tenían trabajo en la semana de referencia.',
    'POCUPADA_F' : 'Población femenina ocupada: Mujeres de 12 a 130 años de edad que trabajaron o que no trabajaron pero sí tenían trabajo en la semana de referencia.',
    'PDESOCUP' : 'Población desocupada: Personas de 12 a 130 años de edad que no tenían trabajo, pero buscaron trabajo en la semana de referencia. ',
    'PDESOCUP_M' : 'Población masculina desocupada: Hombres de 12 a 130 años de edad que no tenían trabajo, pero buscaron trabajo en la semana de referencia.',
    'PDESOCUP_F' : 'Población femenina desocupada: Mujeres de 12 a 130 años de edad que no tenían trabajo, pero buscaron trabajo en la semana de referencia.',
    'PSINDER' : 'Población sin derechohabiencia a servicios de salud  : Total de personas que no tienen derecho a recibir servicios médicos en ninguna institución pública o privada.',
    'PDER_SS ' : 'Población derechohabiente a servicios de salud: Total de personas que tienen derecho a recibir servicios médicos en alguna institución de salud pública o privada como: el instituto mexicano del seguro social (imss), el instituto de seguridad y servicios sociales de los trabajadores del estado (issste e issste estatal), petróleos mexicanos (pemex), la secretaría de la defensa nacional (sedena), la secretaría de marina armada de méxico (semar), el sistema de protección social en salud o en otra.',
    'PDER_IMSS ' : 'Población derechohabiente del imss  : Total de personas que tienen derecho a recibir servicios médicos en el instituto mexicano del seguro social (imss).',
    'PDER_ISTE' : 'Población derechohabiente del issste: Total de personas que tienen derecho a recibir servicios médicos en el instituto de seguridad y servicios sociales de los trabajadores del estado.',
    'PDER_ISTEE' : 'Población derechohabiente del issste estatal: Total de personas que tienen derecho a recibir servicios médicos en los institutos de seguridad social de los estados (issset, isssemym, issstezac, issspea o isssteson).',
    'PDER_SEGP' : 'Población derechohabiente del seguro popular o seguro médico para una nueva generación: Total de personas que tienen derecho a recibir servicios médicos en la secretaría de salud, mediante el sistema de protección en salud (seguro popular).',
    'P12YM_SOLT' : 'Población soltera o nunca unida de 12 años y más: Personas solteras de 12 a 130 años de edad. ',
    'P12YM_CASA' : 'Población casada o unida de 12 años y más : Personas de 12 a 130 años de edad que viven con su pareja en unión libre; casadas solo por el civil; casadas solo religiosamente o; casadas por el civil y religiosamente. ',
    'P12YM_SEPA' : 'Población que estuvo casada o unida de 12 años y más : Personas de 12 a 130 años de edad que están separadas, divorciadas o viudas. ',
    'PCATOLICA' : 'Población con religión católica : Personas con religión católica. ',
    'PNCATOLICA' : 'Protestantes, evangélicas y bíblicas diferentes de evangélicas: Personas con religiones protestantes históricas, pentecostales, neopentecostales, iglesia del dios vivo, columna y apoyo de la verdad, la luz del mundo, cristianas,  evangélicas y bíblicas diferentes de las evangélicas.',
    'POTRAS_REL' : 'Población con otras religiones diferentes a las anteriores: Personas con religiones de origen oriental, judaico, islámico, new age, escuelas esotéricas, raíces étnicas, espiritualistas, ortodoxos,  otros movimientos religiosos y cultos populares.',
    'PSIN_RELIG' : 'Población sin religión: Personas sin adscripción religiosa. Incluye ateísmo. ',
    'TOTHOG' : 'Total de hogares censales: Hogares en viviendas particulares habitadas. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada. ',
    'HOGJEF_M' : 'Hogares censales con jefatura masculina: Hogares en viviendas particulares habitadas donde el jefe es hombre. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'HOGJEF_F' : 'Hogares censales con jefatura femenina: Hogares en viviendas particulares habitadas donde el jefe es mujer. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'POBHOG' : 'Población en hogares censales: Personas en hogares censales. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'PHOGJEF_M' : 'Población en hogares censales con jefatura masculina: Personas en hogares censales donde el jefe es hombre. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'PHOGJEF_F' : 'Población en hogares censales con jefatura femenina: Personas en hogares censales donde el jefe es mujer. Se considera un hogar en cada vivienda particular. Incluye casa independiente; departamento en edificio; vivienda en vecindad; vivienda en cuarto de azotea;  local no construido para habitación; vivienda móvil; refugio o clase no especificada.',
    'VIVTOT' : 'Total de viviendas: Viviendas particulares habitadas, deshabitadas, de uso temporal y colectivas. Incluye a las viviendas particulares sin información de sus ocupantes.',
    'TVIVHAB' : 'Total de viviendas habitadas: Viviendas particulares y colectivas habitadas. Incluye a las viviendas particulares sin información de sus ocupantes.',
    'TVIVPAR' : 'Total de viviendas particulares: Viviendas particulares habitadas, deshabitadas y de uso temporal. Excluye a las viviendas particulares sin información de ocupantes.',
    'VIVPAR_HAB' : 'Viviendas particulares habitadas: Viviendas particulares habitadas de cualquier clase: casa independiente, departamento en edificio, vivienda o cuarto en vecindad, vivienda o cuarto de azotea, local no construido para habitación, vivienda móvil, refugios o clase no especificada. Excluye a las viviendas particulares sin información de ocupantes.',
    'TVIVPARHAB' : 'Total de viviendas particulares habitadas: Viviendas particulares habitadas de cualquier clase: casa independiente, departamento en edificio, vivienda o cuarto en vecindad, vivienda o cuarto de azotea, local no construido para habitación, vivienda móvil, refugios o clase no especificada Incluye a las viviendas particulares sin información de ocupantes.',
    'VIVPAR_DES' : 'Viviendas particulares deshabitadas: Viviendas particulares deshabitadas.',
    'VIVPAR_UT' : 'Viviendas particulares de uso temporal: Viviendas particulares de uso temporal.',
    'OCUPVIVPAR' : 'Ocupantes en viviendas particulares habitadas: Personas que residen en viviendas particulares habitadas de cualquier clase: casa independiente, departamento en edificio, vivienda o cuarto en vecindad, vivienda o cuarto de azotea, local no construido para habitación, vivienda móvil, refugios o clase no especificada. Excluye la estimación del número de personas en viviendas particulares sin información de ocupantes.',
    'PROM_OCUP' : 'Promedio de ocupantes en viviendas particulares habitadas: Resultado de dividir el número de personas que residen en viviendas particulares habitadas, entre el número de esas viviendas. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'PRO_OCUP_C' : 'Promedio de ocupantes por cuarto en viviendas particulares habitadas: Resultado de dividir el número de personas que residen en viviendas particulares habitadas entre el número de cuartos de esas viviendas. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_PISODT' : 'Viviendas particulares habitadas con piso de material diferente de tierra: Viviendas particulares habitadas con piso de cemento o firme, madera, mosaico u otro material. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_PISOTI' : 'Viviendas particulares habitadas con piso de tierra: Viviendas particulares habitadas con piso de tierra. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_1DOR' : 'Viviendas particulares habitadas con un dormitorio: Viviendas particulares habitadas donde sólo uno de los cuartos se usa para dormir. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_2YMASD' : 'Viviendas particulares habitadas con dos dormitorios y más: Viviendas particulares habitadas que usan para dormir entre 2 y 25 cuartos. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_1CUART' : 'Viviendas particulares habitadas con un solo cuarto : Viviendas particulares habitadas que tienen un solo cuarto. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'VPH_2CUART' : 'Viviendas particulares habitadas con dos cuartos: Viviendas particulares habitadas que tienen dos cuartos. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'VPH_3YMASC ' : 'Viviendas particulares habitadas con 3 cuartos y más: Viviendas particulares habitadas que tienen entre 3 y 25 cuartos. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_C_ELEC' : 'Viviendas particulares habitadas que disponen de luz eléctrica: Viviendas particulares habitadas que tienen luz eléctrica. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_S_ELEC' : 'Viviendas particulares habitadas que no disponen de luz eléctrica: Viviendas particulares habitadas que no tienen luz eléctrica. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_AGUADV' : 'Viviendas particulares habitadas que disponen de agua entubada en  el ámbito de la vivienda: Viviendas particulares habitadas que tienen disponibilidad de agua entubada dentro de la vivienda, o fuera de la vivienda pero dentro del terreno. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. ',
    'VPH_AGUAFV' : 'Viviendas particulares habitadas que no disponen de agua entubada en el ámbito de la vivienda: Viviendas particulares habitadas que tienen disponibilidad de agua de una llave pública o hidrante, de otra vivienda, de pipa, de pozo, río, arroyo, lago u otro. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_EXCSA' : 'Viviendas particulares habitadas que disponen de excusado o sanitario: Viviendas particulares habitadas que tienen excusado, retrete, sanitario, letrina u hoyo negro. Excluye la estimación del número de personas y de viviendas particulares sin información de ocupantes.',
    'VPH_DRENAJ' : 'Viviendas particulares habitadas que disponen de drenaje: Viviendas particulares habitadas que tienen drenaje conectado a la red pública, fosa séptica, barranca, grieta, río, lago o mar. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. ',
    'VPH_NODREN' : 'Viviendas particulares habitadas que no disponen de drenaje: Viviendas particulares habitadas que no tienen drenaje. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda. ',
    'VPH_C_SERV' : 'Viviendas particulares habitadas que disponen de luz eléctrica, agua entubada de la red pública y drenaje : Viviendas particulares habitadas que tienen luz eléctrica, agua entubada dentro o fuera de la vivienda, pero dentro del terreno, así como drenaje. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_SNBIEN' : 'Viviendas particulares habitadas sin ningún bien: Viviendas particulares habitadas que no disponen de radio, televisión, refrigerador, lavadora, automóvil, computadora, teléfono fijo, celular ni internet. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_RADIO' : 'Viviendas particulares habitadas que disponen de radio: Viviendas particulares habitadas que tienen radio. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_TV' : 'Viviendas particulares habitadas que disponen de televisor: Viviendas particulares habitadas que tienen televisor. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_REFRI' : 'Viviendas particulares habitadas que disponen de refrigerador: Viviendas particulares habitadas que tienen refrigerador. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_LAVAD' : 'Viviendas particulares habitadas que disponen de lavadora: Viviendas particulares habitadas que tienen lavadora. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_AUTOM' : 'Viviendas particulares habitadas que disponen de automóvil o camioneta: Viviendas particulares habitadas que tienen automóvil o camioneta. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_PC ' : 'Viviendas particulares habitadas que disponen de computadora: Viviendas particulares habitadas que tienen computadora. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_TELEF' : 'Viviendas particulares habitadas que disponen de línea telefónica fija: Viviendas particulares habitadas que tienen línea telefónica fija. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_CEL' : 'Viviendas particulares habitadas que disponen de teléfono celular: Viviendas particulares habitadas que tienen teléfono celular. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
    'VPH_INTER' : 'Viviendas particulares habitadas que disponen de internet: Viviendas particulares habitadas que tienen servicio de internet. Comprende las viviendas particulares para las que se captaron las características de la vivienda, clasificadas como casa independiente, departamento en edificio, vivienda o cuarto en vecindad y vivienda o cuarto en azotea y a las que no especificaron clase de vivienda.',
}

In [14]:
len(Variables)

197

In [15]:
# Metadatos a dataframe para exportar
c2010meta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
c2010meta.columns = ['Descripcion']
c2010meta = c2010meta.rename_axis('Metadato')
c2010meta

,Descripcion
Metadato,
Nombre del Dataset,Censo de Población y Vivienda 2010
Descripcion del dataset,None
Disponibilidad Temporal,2010
Periodo de actualizacion,Decada
Nivel de Desagregacion,Localidad
Notas,s/n
Fuente,INEGI (Microdatos)
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/ccpv/20...
Dataset base,None


In [50]:
# Variables a dataframe para exportar
c2010vars = pd.DataFrame.from_dict(Variables, orient='index', dtype=None)
c2010vars.columns = ['Descripcion']
c2010vars = c2010vars.rename_axis('Mnemonico')
c2010vars.head()

,Descripcion
Mnemonico,
CVE_MUN,Clave geoestadística municipal de 5 dígitos co...
ENTIDAD,Clave de entidad federativa: Clave que identif...
MUN,Clave de municipio ó delegación: Clave que ide...
LOC,Clave de localidad: Clave que identifica a la ...
AGEB,Clave de ageb: Clave que identifica al ageb ur...


In [51]:
# Exportar dataset completo del censo 2010
file = r'D:\PCCS\00_RawData\01_CSV\CPV2010'+'\\'+ 'CPV2010.xlsx'
writer = pd.ExcelWriter(file)
print('Escribiendo Metadatos ... ')
c2010meta.to_excel(writer, sheet_name = 'METADATOS')
print('Escribiendo Variables ... ')
c2010vars.to_excel(writer, sheet_name = 'VARIABLES')
print('Escribiendo Censo ... ')
censo2010.to_excel(writer, sheet_name = 'CENSO2010')
writer.save()
print('---------------TERMINADO---------------')

Escribiendo Metadatos ... 
Escribiendo Variables ... 
Escribiendo Censo ... 
---------------TERMINADO---------------


Posteriormente es necesario guardar únicamente las variables relacionadas de vivienda. Las variables relacionadas a vivienda son las siguientes:

In [16]:
viv_vars = ['VIVTOT', 'TVIVHAB', 'TVIVPAR', 'VIVPAR_HAB', 'TVIVPARHAB', 'VIVPAR_DES']
viv_vars = {k: Variables[k] for k in viv_vars}
for variable, descripcion in viv_vars.items():
    print('{} - {}\n'.format(variable, descripcion))

VIVTOT - Total de viviendas: Viviendas particulares habitadas, deshabitadas, de uso temporal y colectivas. Incluye a las viviendas particulares sin información de sus ocupantes.

TVIVHAB - Total de viviendas habitadas: Viviendas particulares y colectivas habitadas. Incluye a las viviendas particulares sin información de sus ocupantes.

TVIVPAR - Total de viviendas particulares: Viviendas particulares habitadas, deshabitadas y de uso temporal. Excluye a las viviendas particulares sin información de ocupantes.

VIVPAR_HAB - Viviendas particulares habitadas: Viviendas particulares habitadas de cualquier clase: casa independiente, departamento en edificio, vivienda o cuarto en vecindad, vivienda o cuarto de azotea, local no construido para habitación, vivienda móvil, refugios o clase no especificada. Excluye a las viviendas particulares sin información de ocupantes.

TVIVPARHAB - Total de viviendas particulares habitadas: Viviendas particulares habitadas de cualquier clase: casa independie

In [17]:
type(censo2010['TVIVPAR'].loc['01001'].iloc[0])

str

In [18]:
# Debido a que algunas variables contienen caracteres en columnas que deberían ser números, 
# es necesario hacerles varios procesos antes de agregar la información.
# Primero, se eliminan los renglones que están forzando el dtype de la columna a 'str'.
# Luego se cambia el tipo de columna de 'str' a 'int'
# Y por último se hace la agregación

def aggfunc(x):
    return x.replace({'*' : None, 'N/D' : None}).dropna().astype('int').groupby(level=0).agg('sum')

La información se agrega a nivel CVE_MUN.

In [19]:
# Agregar datos de vivienda a nivel CVE_MUN y construir nuevo dataset
vivienda = censo2010.groupby(level=0).agg('sum')['VIVTOT'].to_frame(name = 'VIVTOT')
vivienda['TVIVHAB'] = censo2010.groupby(level=0).agg('sum')['TVIVHAB']
vivienda['TVIVPAR'] = aggfunc(censo2010['TVIVPAR'])
vivienda['VIVPAR_HAB'] = aggfunc(censo2010['VIVPAR_HAB'])
vivienda['TVIVPARHAB'] = aggfunc(censo2010['TVIVPARHAB'])
vivienda['VIVPAR_DES'] = aggfunc(censo2010['VIVPAR_DES'])

In [20]:
vivienda.head()

,VIVTOT,TVIVHAB,TVIVPAR,VIVPAR_HAB,TVIVPARHAB,VIVPAR_DES
CVE_MUN,,,,,,
01001,247792,202141,246166,200684,201666,36809
01002,11906,10012,11756,9888,9892,1410
01003,16676,12847,16491,12755,12765,2466
01004,3996,3467,3933,3425,3427,383
01005,30199,22924,29770,22660,22771,5779


In [21]:
len(vivienda)

2456

Por ultimo, se guardan los datos en un archivo xlsx para construir parámetros.

In [22]:
# Variables a dataframe para exportar
viv_vars = pd.DataFrame.from_dict(viv_vars, orient='index', dtype=None)
viv_vars.columns = ['Descripcion']
viv_vars = viv_vars.rename_axis('Mnemonico')
viv_vars.head()

,Descripcion
Mnemonico,
VIVTOT,Total de viviendas: Viviendas particulares hab...
TVIVHAB,Total de viviendas habitadas: Viviendas partic...
TVIVPAR,Total de viviendas particulares: Viviendas par...
VIVPAR_HAB,Viviendas particulares habitadas: Viviendas pa...
TVIVPARHAB,Total de viviendas particulares habitadas: Viv...


In [26]:
# Exportar dataset de vivienda
file = r'D:\PCCS\01_Dmine\Datasets\CPV'+'\\'+ 'P0309.xlsx'
writer = pd.ExcelWriter(file)
print('Escribiendo Metadatos ... ')
c2010meta.to_excel(writer, sheet_name = 'METADATOS')
print('Escribiendo Variables ... ')
viv_vars.to_excel(writer, sheet_name = 'VARIABLES')
print('Escribiendo Datos de Vivienda ... ')
vivienda.to_excel(writer, sheet_name = 'P0309')
writer.save()
print('---------------TERMINADO---------------')

Escribiendo Metadatos ... 
Escribiendo Variables ... 
Escribiendo Datos de Vivienda ... 
---------------TERMINADO---------------


Quedó bonito